# Degradation and Acceleration Factors
This tool will provide a simple method for estimating degradation and for calculating acceleration factors. It interfaces with the degradation database to simplify acquisition of the degradation parameters.

**Requirements**:
- compatible weather file (PSM3, TMY3, EPW)
- Accelerated testing chamber parameters
    - chamber irradiance [W/m^2]
    - chamber temperature [C]
    - chamber humidity [%]
    - & etc.
- Activation energies for test material [kJ/mol]
- Other degradation parameters

**Objectives**:
1. Read in the weather data
2. Gather basic degradation modeling data for a material of interest
3. Calculate absolute degradation rate
4. Run Monte Carlo simulation at a single site
5. Generate chamber or field data for environmental comparison
6. Calculate degradation acceleration factor of field location to chamber (or another location)
7. Run Monte Carlo simulation of the acceleration factor for a single site
8. Select a region of interest and specific data points
9. Produce a map of acceleration factors for a region

In [ ]:
# if running on google colab, uncomment the next line and execute this cell to install the dependencies and prevent "ModuleNotFoundError" in later cells:
# !pip install pvdeg==0.4.2

In [ ]:
import os
import pvdeg
import pandas as pd
from pvdeg import DATA_DIR
import numpy as np
import json

In [7]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("pvdeg version ", pvdeg.__version__)
print(DATA_DIR)

Working on a  Windows 10
Python version  3.9.18 (main, Sep 11 2023, 14:09:26) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.1.4
pvdeg version  0.4.3.dev14+g08d739a
C:\Users\mkempe\Documents\GitHub\new\PVDegradationTools\pvdeg\data


## 1. Read In the Weather Data

The function has these minimum requirements when using a weather data file:
- Weather data containing (at least) DNI, DHI, GHI, Temperature, RH, and Wind-Speed data at module level.
- Site meta-data containing (at least) latitude, longitude, and time zone

Alternatively one may can get meterological data from the NSRDB or PVGIS with just the longitude and latitude. This function for the NSRDB (via NSRDB 'PSM3') works primarily for most of North America and South America. PVGIS works for most of the rest of the world (via SARAH 'PVGIS'). See the tutorial "Weather Database Access.ipynb" tutorial on PVdeg or Jensen et al. https://doi.org/10.1016/j.solener.2023.112092 for satellite coverage information.

In [8]:
# Get data from a supplied data file (Do not use the next box of code if using your own file)
weather_file = os.path.join(DATA_DIR, 'psm3_demo.csv')
WEATHER_df, META = pvdeg.weather.read(weather_file,'csv')
print(META)

{'Source': 'NSRDB', 'Location ID': 145809.0, 'City': 'West Pleasant View', 'State': 'Colorado', 'Country': 'United States', 'Clearsky DHI Units': 'w/m2', 'Clearsky DNI Units': 'w/m2', 'Clearsky GHI Units': 'w/m2', 'Dew Point Units': 'c', 'DHI Units': 'w/m2', 'DNI Units': 'w/m2', 'GHI Units': 'w/m2', 'Solar Zenith Angle Units': 'Degree', 'Temperature Units': 'c', 'Pressure Units': 'mbar', 'Relative Humidity Units': '%', 'Precipitable Water Units': 'cm', 'Wind Direction Units': 'Degrees', 'Wind Speed Units': 'm/s', 'Cloud Type -15': 'N/A', 'Cloud Type 0': 'Clear', 'Cloud Type 1': 'Probably Clear', 'Cloud Type 2': 'Fog', 'Cloud Type 3': 'Water', 'Cloud Type 4': 'Super-Cooled Water', 'Cloud Type 5': 'Mixed', 'Cloud Type 6': 'Opaque Ice', 'Cloud Type 7': 'Cirrus', 'Cloud Type 8': 'Overlapping', 'Cloud Type 9': 'Overshooting', 'Cloud Type 10': 'Unknown', 'Cloud Type 11': 'Dust', 'Cloud Type 12': 'Smoke', 'Fill Flag 0': 'N/A', 'Fill Flag 1': 'Missing Image', 'Fill Flag 2': 'Low Irradiance', '

In [11]:
# This routine will get a meteorological dataset from anywhere in the world where it is available 
#weather_id = (24.7136, 46.6753) #Riyadh, Saudi Arabia
#weather_id = (35.6754, 139.65) #Tokyo, Japan
weather_id = (-43.52646, 172.62165) #Christchurch, New Zealand
#weather_id = (64.84031, -147.73836) #Fairbanks, Alaska
#weather_id = (65.14037, -21.91633) #Reykjavik, Iceland
#weather_id = (33.4152, -111.8315) #Mesa, Arizona
#weather_id = (0,0) # Somewhere else you are interested in.
WEATHER_df, META = pvdeg.weather.get_anywhere(id=weather_id)
print(META)

{'latitude': -43.52646, 'longitude': 172.62165, 'altitude': 4.0, 'wind_height': 10, 'Source': 'PVGIS', 'leisure': 'Hagley Golf Club', 'suburb': 'Central City', 'city_district': 'Linwood-Central-Heathcote Community', 'ISO3166-2-lvl4': 'NZ-CAN', 'Street': 'Uni-Cycle Cycleway', 'City': 'Christchurch', 'County': 'Christchurch City', 'State': 'Canterbury', 'Zipcode': '8440', 'Country': 'New Zealand / Aotearoa', 'Country Code': 'NZ'}


## 2. Gather Basic Degradation Modeling Data for a Material of Interest

First we need to gather in the parameters for the degradation process of interest. This includes things such as the activiation energy and parameters defining the sensitivity to moisture, UV light, voltage, and other stressors.
For this tutorial we will need solar position, POA, PV cell and module temperature. Let's gernate those individually with their respective functions.

In [24]:
sol_pos = pvdeg.spectral.solar_position(weather_df=WEATHER, meta=META)

poa_df = pvdeg.spectral.poa_irradiance(weather_df=WEATHER, meta=META, sol_position=sol_pos)

temp_cell = pvdeg.temperature.cell(weather_df=WEATHER, meta=META, poa=poa_df)

temp_module = pvdeg.temperature.module(weather_df=WEATHER, meta=META, poa=poa_df)

The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.


## 3. VantHoff Degradation

Van 't Hoff Irradiance Degradation

For one year of degredation the controlled environmnet lamp settings will need to be set to IWa.

As with most `pvdeg` functions, the following functions will always require two arguments (weather_df and meta)

In [25]:
# chamber irradiance (W/m^2)
I_chamber = 1000
# chamber temperature (C)
temp_chamber = 60

# calculate the VantHoff Acceleration factor
vantHoff_deg = pvdeg.degradation.vantHoff_deg(weather_df=WEATHER, meta=META,
                                              I_chamber=I_chamber,
                                              temp_chamber=temp_chamber,
                                              poa=poa_df,
                                              temp=temp_cell)

# calculate the VantHoff weighted irradiance
irr_weighted_avg_v = pvdeg.degradation.IwaVantHoff(weather_df=WEATHER, meta=META,
                                                   poa=poa_df,
                                                   temp=temp_cell)

## 4. Arrhenius
Calculate the Acceleration Factor between the rate of degredation of a modeled environmnet versus a modeled controlled environmnet

Example: "If the AF=25 then 1 year of Controlled Environment exposure is equal to 25 years in the field"

Equation:
$$ AF = N * \frac{ I_{chamber}^x * RH_{chamber}^n * e^{\frac{- E_a}{k T_{chamber}}} }{ \Sigma (I_{POA}^x * RH_{outdoor}^n * e^{\frac{-E_a}{k T_outdoor}}) }$$

Function to calculate IWa, the Environment Characterization (W/m²). For one year of degredation the controlled environmnet lamp settings will need to be set at IWa.

Equation:
$$ I_{WA} = [ \frac{ \Sigma (I_{outdoor}^x * RH_{outdoor}^n e^{\frac{-E_a}{k T_{outdood}}}) }{ N * RH_{WA}^n * e^{- \frac{E_a}{k T_eq}} } ]^{\frac{1}{x}} $$

In [26]:
# relative humidity within chamber (%)
rh_chamber = 15
# arrhenius activation energy (kj/mol)
Ea = 40

rh_surface = pvdeg.humidity.surface_outside(rh_ambient=WEATHER['relative_humidity'],
                                               temp_ambient=WEATHER['temp_air'],
                                               temp_module=temp_module)

arrhenius_deg = pvdeg.degradation.arrhenius_deg(weather_df=WEATHER, meta=META,
                                                rh_outdoor=rh_surface,
                                                I_chamber=I_chamber,
                                                rh_chamber=rh_chamber,
                                                temp_chamber=temp_chamber,
                                                poa=poa_df,
                                                temp=temp_cell,
                                                Ea=Ea)

irr_weighted_avg_a = pvdeg.degradation.IwaArrhenius(weather_df=WEATHER, meta=META,
                                                    poa=poa_df,
                                                    rh_outdoor=WEATHER['relative_humidity'],
                                                    temp=temp_cell,
                                                    Ea=Ea)

## 5. Quick Method (Degradation)

For quick calculations, you can omit POA and both module and cell temperature. The function will calculate these figures as needed using the available weather data with the default options for PV module configuration.

In [27]:
# chamber settings
I_chamber= 1000
temp_chamber=60
rh_chamber=15

# activation energy
Ea = 40

vantHoff_deg = pvdeg.degradation.vantHoff_deg(weather_df=WEATHER, meta=META,
                                              I_chamber=I_chamber,
                                              temp_chamber=temp_chamber)

irr_weighted_avg_v = pvdeg.degradation.IwaVantHoff(weather_df=WEATHER, meta=META)

The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.


In [28]:
rh_surface = pvdeg.humidity.surface_outside(rh_ambient=WEATHER['relative_humidity'],
                                               temp_ambient=WEATHER['temp_air'],
                                               temp_module=temp_module)

arrhenius_deg = pvdeg.degradation.arrhenius_deg(weather_df=WEATHER, meta=META,
                                                rh_outdoor=rh_surface,
                                                I_chamber=I_chamber,
                                                rh_chamber=rh_chamber,
                                                temp_chamber=temp_chamber,
                                                Ea=Ea)

irr_weighted_avg_a = pvdeg.degradation.IwaArrhenius(weather_df=WEATHER, meta=META,
                                                    rh_outdoor=WEATHER['relative_humidity'],
                                                    Ea=Ea)

The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.


## 6. Solder Fatigue

Estimate the thermomechanical fatigue of flat plate photovoltaic module solder joints over the time range given using estimated cell temperature. Like other `pvdeg` funcitons, the minimal parameters are (weather_df, meta). Running the function with only these two inputs will use default PV module configurations ( open_rack_glass_polymer ) and the 'sapm' temperature model over the entire length of the weather data. 

In [29]:
fatigue = pvdeg.fatigue.solder_fatigue(weather_df=WEATHER, meta=META)

The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.


If you wish to reduce the span of time or use a non-default temperature model, you may specify the parameters manually. Let's try an explicit example.
We want the solder fatigue estimated over the month of June for a roof mounted glass-front polymer-back module.

1. Lets create a datetime-index for the month of June.
2. Next, generate the cell temperature. Make sure to explicity restrict the weather data to our dt-index for June. Next, declare the PV module configuration.
3. Calculate the fatigue. Explicity specify the time_range (our dt-index for June from step 1) and the cell temperature as we caculated in step 2

In [30]:
# select the month of June
time_range = WEATHER.index[WEATHER.index.month == 6]

# calculate cell temperature over our selected date-time range.
# specify the module configuration
temp_cell = pvdeg.temperature.cell(weather_df=WEATHER.loc[time_range], meta=META,
                                   temp_model='sapm',
                                   conf='insulated_back_glass_polymer')


fatigue = pvdeg.fatigue.solder_fatigue(weather_df=WEATHER, meta=META,
                                       time_range = time_range,
                                       temp_cell = temp_cell)

The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
